In [3]:
import os
import pandas as pd
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
from itertools import combinations

os.chdir('/home/lewis/PhD/compounds-research')
from compounds_research.compound import utils
from compounds_research import settings
from compounds_research.compound.fetch_exchange_rates import make_exchange_rate_df

In [2]:
tokens = []
for k in settings.COMPOUND_TOKEN_TRANSLATOR:
    tokens.append(settings.COMPOUND_TOKEN_TRANSLATOR[k])

In [31]:
# Return dictionaries for where UIP holds and where it doesn't hold
regression_results = {}
regression_results['pair'] = []
regression_results['alpha'] = []
regression_results['beta'] = []
regression_results['r-squared'] = []
regression_results['p-value_alpha'] = []
regression_results['p-value_beta'] = []
regression_results['f-test-p-value'] = []

UIP_holds = {}
UIP_not_holds = {}
for market in combinations(tokens, 2):
    quote_token = market[1]
    base_token = market[0]
    try:
        df = utils.make_ex_rate_and_interest_rate_df(quote_token=quote_token, 
                                                    base_token=base_token, 
                                                    rate_type='borrow_rates', 
                                                    frequency='M', 
                                                    limit=1000)
        lag_length = int(len(df)**0.25)
    except:
        print('Error making dataframe for ' + str(quote_token) + ' to ' + str(base_token))
        continue
    
    df['i_rate_diff'] = df[quote_token] - df[base_token]
    df['ex_rate_diff'] = df[str(base_token)+ '_' + str(quote_token)].diff().shift(-1)
    df = df.dropna()
    model = OLS(endog=df['ex_rate_diff'], exog=sm.add_constant(df['i_rate_diff']))
    res = model.fit(cov_type='HAC', cov_kwds={'maxlags':lag_length})
    # print(res.summary())

    regression_results['pair'].append(base_token + '_' + quote_token)
    regression_results['pair'].append(' ')
    
    regression_results['alpha'].append("{:.2f}".format(res.params[0]))
    regression_results['alpha'].append('(' + str("{:.2f}".format(res.bse[0])) + ')')

    regression_results['beta'].append(res.params[1])
    regression_results['beta'].append('(' + str("{:.2f}".format(res.bse[1])) + ')')

    regression_results['r-squared'].append("{:.2f}".format(res.rsquared))
    regression_results['r-squared'].append(' ')

    regression_results['p-value_alpha'].append("{:.2f}".format(res.pvalues[0]))
    regression_results['p-value_alpha'].append(' ')

    regression_results['p-value_beta'].append("{:.2f}".format(res.pvalues[1]))
    regression_results['p-value_beta'].append(' ')

    hypotheses = '(const = 0), (i_rate_diff = 1)'
    f_test = res.f_test(hypotheses)

    regression_results['f-test-p-value'].append("{:.2f}".format(f_test.pvalue))
    regression_results['f-test-p-value'].append(' ')

    if f_test.pvalue >= 0.05:
        UIP_holds[str(base_token)+ '_' + str(quote_token)] = f_test.pvalue
    else:
        UIP_not_holds[str(base_token)+ '_' + str(quote_token)] = f_test.pvalue
            

In [32]:
df_results = pd.DataFrame(regression_results)
print(df_results.to_latex(index=False))


\begin{tabular}{lllllll}
\toprule
      pair &      alpha &         beta & r-squared & p-value\_alpha & p-value\_beta & f-test-p-value \\
\midrule
   eth\_bat &    -304.80 &      20810.2 &      0.35 &          0.02 &         0.01 &           0.03 \\
           &   (127.13) &    (7618.33) &           &               &              &                \\
   eth\_zrx &      97.47 &      -4824.5 &      0.08 &          0.24 &         0.18 &           0.43 \\
           &    (83.49) &    (3582.73) &           &               &              &                \\
  eth\_usdc &      52.89 &     -631.236 &      0.11 &          0.00 &         0.02 &           0.03 \\
           &    (17.26) &     (263.25) &           &               &              &                \\
   eth\_dai &      20.23 &     -197.874 &      0.01 &          0.23 &         0.74 &           0.43 \\
           &    (16.71) &     (589.31) &           &               &              &                \\
   eth\_sai &       5.52 &     -7